In [ ]:
import pickle
import itertools
import glob
import gc

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter
from pathlib import Path
from random import randint

from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange

from pyecsca.ec.params import get_params
from pyecsca.ec.mult import *
from pyecsca.misc.utils import TaskExecutor

from common import *

%matplotlib ipympl

In [ ]:
# Setup the ticks and colors deterministically.
mult_klasses = sorted(list(set(map(lambda mult: mult.klass, all_mults))), key=lambda klass: klass.__name__)
mult_kwarg_map = {klass: 0 for klass in mult_klasses}
mult_colors = matplotlib.cm.tab20(range(len(mult_klasses)))
mult_styles = ['-', '--', '-.', ':', (5, (10, 3)), (0, (5, 1)), (0, (3, 1, 1, 1, 1, 1)), (0, (3, 1, 1, 1)), (0, (1, 1)), (0, (3, 10, 1, 10))]
colors = {}
styles = {}
for mult in all_mults:
    colors[mult] = mult_colors[mult_klasses.index(mult.klass)]
    styles[mult] = mult_styles[mult_kwarg_map[mult.klass]]
    mult_kwarg_map[mult.klass] += 1

In [ ]:
def divides_any(l: int, small_scalars: set[int]) -> bool:
    for s in small_scalars:
        if s%l==0:
            return True
    return False

def process_small_scalars(scalar_results: MultResults, divisors: set[int]) -> ProbMap:
    result = {}
    for divisor in tqdm(divisors, leave=False):
        count = 0
        for smult in scalar_results.multiplications:
            if divides_any(divisor, smult):
                count += 1
        result[divisor] = count / scalar_results.samples
    return ProbMap(result, scalar_results.samples)

In [ ]:
def powers_of(k, max_power=20):
    return [k**i for i in range(1, max_power)]

def prod_combine(one, other):
    return [a * b for a, b in itertools.product(one, other)]

small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
medium_primes = [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
large_primes = [401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
all_integers = list(range(1, 400))
all_even = list(range(2, 400, 2))
all_odd = list(range(1, 400, 2))
all_primes = small_primes + medium_primes + large_primes

divisor_map = {
    "small_primes": small_primes,
    "medium_primes": medium_primes,
    "large_primes": large_primes,
    "all_primes": all_primes,
    "all_integers": all_integers,
    "all_even": all_even,
    "all_odd": all_odd,
    "powers_of_2": powers_of(2),
    "powers_of_2_large": powers_of(2, 130),
    "powers_of_2_large_p1": [i+1 for i in powers_of(2, 130)],
    "powers_of_3": powers_of(3),
}
divisor_map["all"] = list(sorted(set().union(*[v for v in divisor_map.values()])))

In [ ]:
category = "secg"
curve = "secp256r1"
params = get_params(category, curve, "projective")
bits = params.order.bit_length()
num_workers = 10

In [ ]:
selected_mults = all_mults
divisor_name = "powers_of_2_large_p1"
selected_divisors = divisor_map[divisor_name]

In [ ]:
distributions_mults = {}
files = tqdm(glob.glob(f"multiples_{category}_{curve}_{bits}_chunk*.pickle"))
with TaskExecutor(max_workers=num_workers) as pool:
    for fname in files:
        files.set_postfix_str(fname)
        with open(fname, "rb") as f:
            multiples_loaded = pickle.load(f)
            for mult, results in multiples_loaded.items():
                pool.submit_task(mult,
                                 process_small_scalars,
                                 results, selected_divisors)
        for mult, future in tqdm(pool.as_completed(), leave=False, total=len(pool.tasks)):
            prob_map = future.result()
            if mult in distributions_mults:
                distributions_mults[mult].merge(prob_map)
            else:
                distributions_mults[mult] = prob_map
        del multiples_loaded
        gc.collect()

In [ ]:
plot_mults = selected_mults
plot_divisors = selected_divisors

fig = plt.figure(figsize=(58, 12))
ax = plt.subplot(111)

L = len(plot_divisors)
N = len(plot_mults)
plot_divisors = sorted(plot_divisors)
vals = np.zeros((N, L))
for i, mult in enumerate(plot_mults):
    y_values = [distributions_mults[mult][l] for l in plot_divisors]
    vals[i,] = y_values
    ax.plot(list(range(L)), y_values, color=colors[mult], linestyle=styles[mult], label=str(mult))
    ax.set_title(divisor_name + "\nSamples: " + str(distributions_mults[mult].samples))

var = np.var(vals, axis=0)
#ax.plot(list(range(L)), var / np.max(var), label="cross-mult variance (normalized)", ls="--", lw=2, color="black")

ax.set_xlabel("divisors") 
ax.set_ylabel("error probability")
ax.set_xticks(list(range(L)), plot_divisors, rotation=90)

ax.grid()
plt.tight_layout()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()
fig.savefig(f"graphs/{divisor_name}.png",dpi=300);

In [ ]:
# general_distributions = get_general_distributions(selected_divisors, bits, samples)
# general_n_distributions = get_general_n_distributions(selected_divisors, bits, 256, samples)

In [ ]:
plot_mults = [mult for mult in selected_mults if mult.klass not in (CombMultiplier, BGMWMultiplier)]
plot_divisors = selected_divisors

colors = {mult:matplotlib.cm.tab20(range(len(plot_mults)))[i] for i,mult in enumerate(plot_mults)}

fig = plt.subplots(figsize=(36, 12))

L = len(plot_divisors)
N = len(plot_mults)
plot_divisors = sorted(plot_divisors)
vals = np.zeros((N, L))
for i, mult in enumerate(plot_mults):
    for j, m in enumerate(plot_divisors):
        y = distributions_mults[mult][m]
        vals[i, j] = y
var = np.var(vals, axis=0)
plt.plot(list(range(L)), var, label="cross-mult variance", ls="--", lw=2, color="black")

plt.xlabel("divisors")
plt.ylabel("variance")
plt.xticks(list(range(L)), plot_divisors, rotation=90)

plt.grid()
plt.legend()
plt.show() 
fig[0].savefig(f"graphs/cross_var.png",dpi=300)

In [ ]:

selected_mults = all_mults#window_mults[0:1]+window_mults[5:6]+naf_mults[1:2]#[mult for mult in all_mults if not mult in comb_mults]
selected_divisors = all_divisors
colors = {mult:matplotlib.cm.tab20(range(len(distributions_mults_10k)))[i] for i,mult in enumerate(distributions_mults_10k)}


fig = plt.subplots(figsize =(36, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in distributions_mults_10k:
    y_values = [distributions_mults_10k[mult][l] for l in selected_divisors]
    plt.plot([l for l in range(L)],y_values,color = colors[mult], label = mult_label(mult))

# mult = list(fixedwindow_dist.keys())[0]
# plt.plot([l for l in range(L)],[fixedwindow_dist[l] for l in selected_divisors],color = "pink", label = mult_label(mult))

# measured_dist = measured_distribution(library,selected_divisors)
# mes_x, mes_y = [],[]
# for i,l in enumerate(selected_divisors):
#     if l in measured_dist:
#         mes_y.append(measured_dist[l])
#         mes_x.append(i)
# plt.scatter(mes_x,mes_y,color = "black", label = library)

#attempts = 0
#fails =0
#for i in range(51):
#    with open(f"cards/jcop/199_{i}.txt") as f:
#        attempts += f.read().count("ALG_EC_SVDP_DH of remote pubkey and local privkey")+1
#        fails += 1
#plt.scatter([selected_divisors.index(199)],[fails/attempts],s=[40],color = "black", label = "jcop")

#plt.plot([l for l in range(L)],[general_distributions[l] for l in selected_divisors],color = "black", label = "prime-distribution")


plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)

plt.legend()
plt.show() 
fig[0].savefig(f"graphs/re.png",dpi=300)


In [ ]:
selected_mults = with_precomputation
selected_divisors = all_divisors
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}


fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    plt.plot([l for l in range(L)],[distributions_mults_precomp[mult][l] for l in selected_divisors],color = colors[mult], label = mult_label(mult))


#measured_dist = measured_distribution(library,selected_divisors)
#mes_x, mes_y = [],[]
#for i,l in enumerate(selected_divisors):
#    if l in measured_dist:
#        mes_y.append(measured_dist[l])
#        mes_x.append(i)
#plt.scatter(mes_x,mes_y,color = "black", label = library)


plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)

plt.legend()
plt.show() 

In [ ]:
def nok_ecdh(line):
    return int(line.split(";")[-1].strip(),16)==0

def measured_distribution(library, selected_divisors):
    measured_distribution = {}
    counts = {order:0 for order in selected_divisors}
    for div in selected_divisors:
        errors = 0
        with open(f"./ecdh/{library}/ecdh_{div}.txt") as f:
            for line in f.readlines()[1:]:
                if nok_ecdh(line):
                    errors+=1
                counts[div]+=1
        measured_distribution[div] = errors
    
    for o,v in measured_distribution.items():
        if counts[o]!=0:
            measured_distribution[o] = v/counts[o]
    return measured_distribution

In [ ]:
selected_mults = other_mults[:1]+binary_mults[:1]+comb_mults[:1]+window_mults[:1]
selected_divisors = small_primes#all_divisors
library = "tomcrypt"
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}

fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    plt.plot([l for l in range(L)],[distributions_mults[mult][l] for l in selected_divisors],color = colors[mult], label = mult_label(mult))


measured_dist = measured_distribution(library,selected_divisors)
mes_x, mes_y = [],[]
for i,l in enumerate(selected_divisors):
    if l in measured_dist:
        mes_y.append(measured_dist[l])
        mes_x.append(i)
plt.scatter(mes_x,mes_y,color = "black", label = library)


plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)
plt.legend(loc="upper right")
plt.show() 
fig[0].savefig(f"graphs/{library}/re.png",dpi=300)

In [ ]:
selected_mults = [mult for mult in with_precomputation if mult in comb_mults]
selected_divisors = small_primes#all_divisors
library = "mbedtls"
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}

fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    plt.plot([l for l in range(L)],[distributions_mults_precomp[mult][l] for l in selected_divisors],color = colors[mult], label = mult_label(mult))


measured_dist = measured_distribution(library,selected_divisors)
mes_x, mes_y = [],[]
for i,l in enumerate(selected_divisors):
    if l in measured_dist:
        mes_y.append(measured_dist[l])
        mes_x.append(i)
plt.scatter(mes_x,mes_y,color = "black", label = library)

plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)
plt.legend()
plt.show() 
fig[0].savefig(f"graphs/{library}/re.png",dpi=300)

BouncyCastle
 - WindowBooth-5?

Mbedtls
 - CombMultiplier-4
 - confirmed in library
   
tomcrypt
 - ladder or coron
 - ladder confirmed in library

OpenSSL, LibreSSL, Botan, Crypto++ and IPPCP followed the general distribution of divisibility by the primes. So they have some countermeasure.

Note that some libraries for some orders output "Invalid algorithm parameter: Not supported.". For libcrypt, SunEC and Nettle it happened for all orders.
BoringSSL
 - "Invalid algorithm parameter: Error creating EC_GROUP, EC_GROUP_set_generator."


In [ ]:
def scatter(library, color):
    measured_dist = measured_distribution(library,selected_divisors)
    mes_x, mes_y = [],[]
    for i,l in enumerate(selected_divisors):
        if l in measured_dist:
            mes_y.append(measured_dist[l])
            mes_x.append(i)
    plt.scatter(mes_x,mes_y,color = color, label = library)

selected_divisors = small_primes#all_divisors

fig = plt.subplots(figsize =(24, 12)) 

L = len(selected_divisors)

colors = matplotlib.cm.tab20(range(6))
scatter("openssl",colors[0])
scatter("libressl",colors[1])
scatter("botan",colors[2])
scatter("Crypto++",colors[3])
scatter("ippcp",colors[4])

plt.plot([l for l in range(L)],[general_distributions[l] for l in selected_divisors],color = colors[5], label = "prime-distribution")

plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.xticks([r for r in range(L)], selected_divisors)
plt.legend()
plt.show() 
fig[0].savefig(f"graphs/resistant.png",dpi=300)

In [ ]:
def scatter(library, color):
    measured_dist = measured_distribution(library,selected_divisors)
    mes_x, mes_y = [],[]
    for i,l in enumerate(selected_divisors):
        if l in measured_dist:
            mes_y.append(measured_dist[l])
            mes_x.append(i)
    # plt.scatter(mes_x,mes_y,color = color, label = library)
    plt.plot(mes_x,mes_y,color = color, linewidth = 1, label = library)

selected_divisors = small_primes[:12]#all_divisors

fig = plt.subplots(figsize =(10, 4)) 

L = len(selected_divisors)

colors = matplotlib.cm.tab20(range(9))
scatter("openssl",colors[0])
scatter("libressl",colors[1])
scatter("botan",colors[2])
scatter("Crypto++",colors[3])
scatter("mbedtls",colors[4])
scatter("libressl",colors[5])
scatter("BouncyCastle",colors[6])
scatter("tomcrypt",colors[7])
scatter("ippcp",colors[8])


# plt.plot([l for l in range(L)],[general_distributions[l] for l in selected_divisors],color = colors[9], label = "divison-distribution")

plt.xlabel('Input point order',fontsize=15) 
plt.ylabel("Error rate",fontsize=15) 
plt.xticks([r for r in range(L)], [v if i%1==0 else "" for i,v in enumerate(selected_divisors)])
plt.legend(loc="center right",prop={'size': 11})
plt.tight_layout()
plt.show() 
fig[0].savefig(f"graphs/lib_dists.png",dpi=300)

In [ ]:
def bars(library, color,shift,width):
    measured_dist = measured_distribution(library,selected_divisors)
    mes_x, mes_y = [],[]
    for i,l in enumerate(selected_divisors):
        offset = width*shift
        if l in measured_dist:
            mes_y.append(measured_dist[l])
            mes_x.append(2*i+offset)
    plt.bar(mes_x,mes_y,width=0.1,color = color,align ='center', label = labels.get(library,library))

selected_divisors = small_primes[:12]#all_divisors

fig = plt.subplots(figsize =(10, 4)) 

L = len(selected_divisors)

colors = matplotlib.cm.tab20(range(9))
width = 0.2
for i,lib in enumerate(("openssl","libressl","botan","Crypto++","mbedtls","BouncyCastle","tomcrypt","ippcp")):
    bars(lib,colors[i],i,width)


plt.plot([2*l+4*width for l in range(L)],[general_distributions[l] for l in selected_divisors],color = colors[8], label = "expected distribution")

plt.xlabel('Input point order',fontsize=15) 
plt.ylabel("Error rate",fontsize=15) 
plt.xticks([2*r+4*width for r in range(L)], [v if i%1==0 else "" for i,v in enumerate(selected_divisors)])
plt.legend(loc="upper right",prop={'size': 11})
plt.tight_layout()
plt.show() 
fig[0].savefig(f"graphs/lib_dists.png",dpi=300)

In [ ]:
from math import sqrt

selected_mults = all_mults#window_mults[0:1]+window_mults[5:6]+naf_mults[1:2]#[mult for mult in all_mults if not mult in comb_mults]
selected_divisors = small_primes#ll_divisors
colors = {mult:matplotlib.cm.tab20(range(len(selected_mults)))[i] for i,mult in enumerate(selected_mults)}


fig = plt.subplots(figsize =(30, 20)) 

L = len(selected_divisors)
selected_divisors = sorted(selected_divisors)
for mult in selected_mults:
    y_values,y_values_mstd, y_values_pstd = [],[],[]
    for l in selected_divisors:
        p = distributions_mults[mult][l]
        y_values.append(1/p)
        y_values_mstd.append(1/p-sqrt((1-p)/p**2))
        y_values_pstd.append(1/p+sqrt((1-p)/p**2))
    plt.plot([l for l in range(L)],y_values,color = colors[mult], label = mult_label(mult))
    plt.fill_between([l for l in range(L)], y_values_mstd , y_values_pstd, alpha = 0.1, color = colors[mult])
    
# mult = list(fixedwindow_dist.keys())[0]
# plt.plot([l for l in range(L)],[fixedwindow_dist[l] for l in selected_divisors],color = "pink", label = mult_label(mult))

# measured_dist = measured_distribution(library,selected_divisors)
# mes_x, mes_y = [],[]
# for i,l in enumerate(selected_divisors):
#     if l in measured_dist:
#         mes_y.append(measured_dist[l])
#         mes_x.append(i)
# plt.scatter(mes_x,mes_y,color = "black", label = library)

plt.plot([l for l in range(L)],[1/general_distributions[l] for l in selected_divisors],color = "black", label = "prime-distribution")

steps_dist = {}
for i in range(51):
    with open(f"cards/jcop/199_{i}.txt") as f:
        steps = f.read().count("ALG_EC_SVDP_DH of remote pubkey and local privkey")
    if not steps in steps_dist:
        steps_dist[steps] = 0
    steps_dist[steps]+=1
ys = sorted(list(steps_dist.keys()))
plt.scatter([selected_divisors.index(199)]*len(ys),ys, s=[steps_dist[y]*20 for y in ys],color="black")
for i, y in enumerate(ys):
    plt.annotate(str(steps_dist[y]), (selected_divisors.index(199)-1, y))
avg = 0
for s,c in steps_dist.items():
    avg+=s*c
avg = avg/sum(steps_dist.values())
plt.scatter([selected_divisors.index(199)],[avg], s=[30],color="yellow")
    
plt.xlabel('divisors') 
plt.ylabel("prob") 
plt.yticks(range(12))
plt.xticks([r for r in range(L)], selected_divisors)

plt.legend()
plt.show() 
fig[0].savefig(f"graphs/re.png",dpi=300)